In [ ]:
!pip install gymnasium

In [ ]:
!pip install torch stable-baselines3 gym numpy
!pip install shimmy

  Using cached gymnasium-0.29.1-py3-none-any.whl.metadata (10 kB)
Using cached gymnasium-0.29.1-py3-none-any.whl (953 kB)
  Attempting uninstall: gymnasium
    Found existing installation: gymnasium 1.0.0a2
    Uninstalling gymnasium-1.0.0a2:
      Successfully uninstalled gymnasium-1.0.0a2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
shimmy 2.0.0 requires gymnasium>=1.0.0a1, but you have gymnasium 0.29.1 which is incompatible.


  Using cached gymnasium-1.0.0a2-py3-none-any.whl.metadata (10 kB)
Using cached gymnasium-1.0.0a2-py3-none-any.whl (954 kB)
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3070, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2863, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self

In [ ]:
# import gym
# from gym import spaces
# import numpy as np

# class WarehouseEnv(gym.Env):
#     metadata = {'render.modes': ['human']}

#     def __init__(self, grid_size=(5, 5), obstacles=None, starts=None, destinations=None):
#         super(WarehouseEnv, self).__init__()

#         self.grid_size = grid_size
#         self.obstacles = obstacles if obstacles else []
#         self.starts = starts if starts else [(0, 0)]
#         self.destinations = destinations if destinations else [(4, 4)]

#         # Define action and observation space
#         # Actions: 0 = Forward, 1 = Reverse, 2 = Left, 3 = Right, 4 = Wait
#         self.action_space = spaces.Discrete(5)

#         # Observation space will be the grid state (you can customize it)
#         self.observation_space = spaces.Box(low=0, high=1, shape=(grid_size[0], grid_size[1], 1), dtype=np.float32)

#         self.grid = self.create_grid()
#         self.agent_positions = None
#         self.commands = []  # Track commands issued to each bot
#         self.time_steps = 0  # Track total time
#         self.reset()

#     def create_grid(self):
#         grid = np.zeros(self.grid_size)
#         for (x, y) in self.obstacles:
#             grid[x, y] = -1  # Mark obstacles as -1
#         return grid

#     def reset(self):
#         # Reset the grid and the positions of autobots
#         self.grid = self.create_grid()
#         self.agent_positions = self.starts.copy()  # Track agent starting positions
#         self.commands = []  # Reset commands
#         self.time_steps = 0  # Reset time steps
#         return np.expand_dims(self.grid, axis=-1)

#     def step(self, actions):
#         if not isinstance(actions, (list, tuple, np.ndarray)):
#             actions = [actions]
#         done = False
#         reward = 0
#         self.time_steps += 1  # Increment time
#         movements = ["Forward", "Reverse", "Left", "Right", "Wait"]

#         for i, action in enumerate(actions):
#             self.commands.append(f"Bot {i+1}: {movements[action]}")
#             # Update positions based on action
#             # Example: move forward, reverse, left, right
#             # Implement position update and reward logic here

#         # For now, return observation, reward, done, and info (customize this as needed)
#         return np.expand_dims(self.grid, axis=-1), reward, done, {}

#     def render(self, mode='human'):
#         # Render the grid and positions of autobots
#         print("Grid:")
#         print(self.grid)
#         print("Commands issued so far:")
#         for command in self.commands:
#             print(command)

#     def final_output(self):
#         # Print the final output as per PDF format
#         print("=== Final Output ===")
#         for i, position in enumerate(self.agent_positions):
#             print(f"Bot {i+1}: Final position at {position}")
#         print(f"Total Time: {self.time_steps}")
#         print(f"Total Commands: {len(self.commands)}")
import gym
from gym import spaces
import numpy as np

class WarehouseEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, grid_size=(5, 5), obstacles=None, starts=None, destinations=None):
        super(WarehouseEnv, self).__init__()

        self.grid_size = grid_size
        self.obstacles = obstacles if obstacles else []
        self.starts = starts if starts else [(0, 0)]
        self.destinations = destinations if destinations else [(4, 4)]

        # Define action and observation space
        # Actions: 0 = Forward, 1 = Reverse, 2 = Left, 3 = Right, 4 = Wait
        self.action_space = spaces.Discrete(5)

        # Observation space will be the grid state (you can customize it)
        self.observation_space = spaces.Box(low=0, high=1, shape=(grid_size[0], grid_size[1], 1), dtype=np.float32)

        self.grid = self.create_grid()
        self.agent_positions = None
        self.agent_directions = []  # Directions each bot is facing (initially facing upward)
        self.commands = []  # Track commands issued to each bot
        self.time_steps = 0  # Track total time
        self.reset()

    def create_grid(self):
        grid = np.zeros(self.grid_size)
        for (x, y) in self.obstacles:
            grid[x, y] = -1  # Mark obstacles as -1
        return grid

    def reset(self):
        # Reset the grid and the positions of autobots
        self.grid = self.create_grid()
        self.agent_positions = self.starts.copy()  # Track agent starting positions
        self.agent_directions = ["up" for _ in self.starts]  # Initially all bots face upwards
        self.commands = []  # Reset commands
        self.time_steps = 0  # Reset time steps
        return np.expand_dims(self.grid, axis=-1)

    def step(self, actions):
        done = False
        reward = 0
        self.time_steps += 1  # Increment time
        movements = ["Forward", "Reverse", "Left", "Right", "Wait"]

        for i, action in enumerate(actions):
            self.commands.append(f"Bot {i+1}: {movements[action]}")
            self.execute_action(i, action)  # Move each bot based on the action

        # Example: check if all bots have reached their destination
        done = all([self.agent_positions[i] == self.destinations[i] for i in range(len(self.starts))])

        # For now, return observation, reward, done, and info (customize this as needed)
        return np.expand_dims(self.grid, axis=-1), reward, done, {}

    def execute_action(self, bot_index, action):
        """
        Execute the action for the bot at the given index, updating its position
        while avoiding obstacles and collisions.
        """
        current_pos = self.agent_positions[bot_index]
        new_pos = list(current_pos)  # Copy current position

        # Determine movement based on current direction and action
        if action == 0:  # Forward
            if self.agent_directions[bot_index] == "up":
                new_pos[0] -= 1  # Move up
            elif self.agent_directions[bot_index] == "down":
                new_pos[0] += 1  # Move down
            elif self.agent_directions[bot_index] == "left":
                new_pos[1] -= 1  # Move left
            elif self.agent_directions[bot_index] == "right":
                new_pos[1] += 1  # Move right
        elif action == 1:  # Reverse
            if self.agent_directions[bot_index] == "up":
                new_pos[0] += 1  # Move down
            elif self.agent_directions[bot_index] == "down":
                new_pos[0] -= 1  # Move up
            elif self.agent_directions[bot_index] == "left":
                new_pos[1] += 1  # Move right
            elif self.agent_directions[bot_index] == "right":
                new_pos[1] -= 1  # Move left
        elif action == 2:  # Left turn
            self.turn_left(bot_index)
        elif action == 3:  # Right turn
            self.turn_right(bot_index)
        elif action == 4:  # Wait
            pass

        # Make sure the new position is within bounds and does not collide
        if self.valid_move(new_pos, bot_index):
            self.agent_positions[bot_index] = tuple(new_pos)

    def turn_left(self, bot_index):
        """Turn the bot 90 degrees to the left."""
        direction = self.agent_directions[bot_index]
        if direction == "up":
            self.agent_directions[bot_index] = "left"
        elif direction == "left":
            self.agent_directions[bot_index] = "down"
        elif direction == "down":
            self.agent_directions[bot_index] = "right"
        elif direction == "right":
            self.agent_directions[bot_index] = "up"

    def turn_right(self, bot_index):
        """Turn the bot 90 degrees to the right."""
        direction = self.agent_directions[bot_index]
        if direction == "up":
            self.agent_directions[bot_index] = "right"
        elif direction == "right":
            self.agent_directions[bot_index] = "down"
        elif direction == "down":
            self.agent_directions[bot_index] = "left"
        elif direction == "left":
            self.agent_directions[bot_index] = "up"

    def valid_move(self, new_pos, bot_index):
        """Check if the new position is within bounds and doesn't collide with obstacles or other bots."""
        if (0 <= new_pos[0] < self.grid_size[0] and
            0 <= new_pos[1] < self.grid_size[1] and
            tuple(new_pos) not in self.obstacles and
            tuple(new_pos) not in [self.agent_positions[j] for j in range(len(self.agent_positions)) if j != bot_index]):
            return True
        return False

    def render(self, mode='human'):
        # Render the grid and positions of autobots
        print("Grid:")
        grid_with_bots = self.grid.copy()
        for i, pos in enumerate(self.agent_positions):
            grid_with_bots[pos] = i + 1  # Mark bots on the grid
        print(grid_with_bots)
        print("Commands issued so far:")
        for command in self.commands:
            print(command)

    def final_output(self):
        # Print the final output as per PDF format
        print("=== Final Output ===")
        for i, position in enumerate(self.agent_positions):
            print(f"Bot {i+1}: Final position at {position}")
        print(f"Total Time: {self.time_steps}")
        print(f"Total Commands: {len(self.commands)}")


In [ ]:
from stable_baselines3.common.env_util import make_vec_env

# # # Create vectorized environment
# # vec_env = make_vec_env(lambda: WarehouseEnv(grid_size=(5, 5), obstacles=[(0, 3), (1, 1)],
# #                                             starts=[(0, 0), (4, 0)],
# #                                             destinations=[(0, 4), (4, 4)]), n_envs=2)
# # Initialize and reset environment with multiple bots
# env = WarehouseEnv(grid_size=(5, 5),
#                    obstacles=[(0, 3), (1, 1), (2, 2)],
#                    starts=[(0, 0), (4, 0), (0, 4)],  # Different start positions for each bot
#                    destinations=[(4, 4), (0, 4), (4, 0)])  # Different end positions

# obs = env.reset()

# # Print initial positions
# print("=== Initial Positions ===")
# for i, start in enumerate(env.starts):
#     print(f"Bot {i+1}: Initial position at {start}")

# # Test loop for evaluation
# for _ in range(10):  # Set a small number of steps for testing
#     actions = [env.action_space.sample() for _ in range(len(env.starts))]  # Random actions for each bot
#     obs, rewards, done, info = env.step(actions)
#     env.render()

# # After the simulation, print the final output
# env.final_output()
# Initialize and reset environment with multiple bots
# env = WarehouseEnv(grid_size=(5, 5),
#                    obstacles=[(0, 3), (1, 1), (2, 2)],
#                    starts=[(0, 0), (4, 0), (0, 4)],  # Different start positions for each bot
#                    destinations=[(4, 4), (0, 4), (4, 0)])  # Different end positions
env = WarehouseEnv(grid_size=(5, 5), obstacles=[(0, 3), (1, 1), (2, 2)],starts=[(0, 0), (4, 0), (0, 4)],destinations=[(4, 4), (0, 4), (4, 0)])
  # Different end positions

obs = env.reset()

# Print initial positions
print("=== Initial Positions ===")
for i, start in enumerate(env.starts):
    print(f"Bot {i+1}: Initial position at {start}")

# Test loop for evaluation
for _ in range(10):  # Set a small number of steps for testing
    actions = [env.action_space.sample() for _ in range(len(env.starts))]  # Random actions for each bot
    obs, rewards, done, info = env.step(actions)
    env.render()

# After the simulation, print the final output
env.final_output()


=== Initial Positions ===
Bot 1: Initial position at (0, 0)
Bot 2: Initial position at (1, 8)
Bot 3: Initial position at (3, 0)
Bot 4: Initial position at (4, 0)
Bot 5: Initial position at (5, 0)
Bot 6: Initial position at (6, 1)
Bot 7: Initial position at (7, 0)
Bot 8: Initial position at (8, 0)
Bot 9: Initial position at (9, 2)
Grid:
[[ 1.  0.  0. -1.  0.  0.  0.  0.  0.  0.]
 [ 0. -1.  0.  0.  0.  0.  0.  0.  2.  0.]
 [ 0.  0. -1.  0.  0.  0.  0.  0.  0.  0.]
 [ 3.  0.  0.  0. -1.  0.  0.  0.  0.  0.]
 [ 4.  0.  0. -1. -1.  0.  0.  0.  0.  0.]
 [ 5.  0.  0. -1.  0.  0.  0.  0.  0.  0.]
 [ 0.  6.  0. -1.  0.  0.  0.  0.  0.  0.]
 [ 7.  0.  0. -1.  0.  0.  0.  0.  0.  0.]
 [ 8.  0.  0. -1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  9.  0.  0.  0.  0.  0.  0.  0.]]
Commands issued so far:
Bot 1: Forward
Bot 2: Right
Bot 3: Reverse
Bot 4: Reverse
Bot 5: Left
Bot 6: Wait
Bot 7: Left
Bot 8: Left
Bot 9: Wait
Grid:
[[ 0.  0.  0. -1.  0.  0.  0.  0.  0.  0.]
 [ 1. -1.  0.  0.  0.  0.  0.  0.  2.  0

In [ ]:
from stable_baselines3 import PPO

# Initialize PPO model (you can customize this to be a MAPPO-like agent)
model = PPO('MlpPolicy', vec_env, verbose=1)

# Train the model for a certain number of timesteps
model.learn(total_timesteps=50000)

# Save the model
model.save("mappo_warehouse")


Using cpu device
-----------------------------
| time/              |      |
|    fps             | 2178 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 4096 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 1166         |
|    iterations           | 2            |
|    time_elapsed         | 7            |
|    total_timesteps      | 8192         |
| train/                  |              |
|    approx_kl            | 0.0125566535 |
|    clip_fraction        | 0.12         |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.6         |
|    explained_variance   | 0            |
|    learning_rate        | 0.0003       |
|    loss                 | -0.0376      |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.00546     |
|    value_loss           | 4.32e-05     |
------------------------------------------

In [ ]:
# Load the model if needed
model = PPO.load("mappo_warehouse")

# Reset the environment
env = WarehouseEnv(grid_size=(5, 5), obstacles=[(0, 3), (1, 1)],
                   starts=[(0, 0), (4, 0)],
                   destinations=[(0, 4), (4, 4)])

obs = env.reset()

# Run a test loop
for _ in range(100):
    actions, _ = model.predict(obs)
    # actions = actions.tolist()
    if not isinstance(actions, list):
      actions = [actions]

    obs, rewards, done, info = env.step(actions)
    env.render()


Streaming output truncated to the last 5000 lines.
Bot 1: Left
Bot 1: Reverse
Grid:
[[ 0.  0.  0. -1.  0.]
 [ 0. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.]
 [ 2.  0.  0.  0.  0.]]
Commands issued so far:
Bot 1: Reverse
Bot 1: Reverse
Bot 1: Right
Bot 1: Left
Bot 1: Forward
Bot 1: Reverse
Bot 1: Reverse
Bot 1: Reverse
Bot 1: Forward
Bot 1: Right
Bot 1: Reverse
Bot 1: Right
Bot 1: Left
Bot 1: Left
Bot 1: Right
Bot 1: Left
Bot 1: Right
Bot 1: Right
Bot 1: Left
Bot 1: Reverse
Bot 1: Wait
Bot 1: Left
Bot 1: Wait
Bot 1: Reverse
Bot 1: Reverse
Bot 1: Reverse
Bot 1: Wait
Bot 1: Right
Bot 1: Right
Bot 1: Forward
Bot 1: Left
Bot 1: Reverse
Bot 1: Reverse
Grid:
[[ 0.  0.  0. -1.  0.]
 [ 0. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.]
 [ 2.  0.  0.  0.  0.]]
Commands issued so far:
Bot 1: Reverse
Bot 1: Reverse
Bot 1: Right
Bot 1: Left
Bot 1: Forward
Bot 1: Reverse
Bot 1: Reverse
Bot 1: Reverse
Bot 1: Forward
Bot 1: Right
Bot 1: Reverse
Bot 1: Right
Bot 1: Lef